# Lab 12 - Python Data Science

Goals

- Loading in a dataset with Python, in this case, every FM radio station licensed in the state of Pennsylvania
- Using Pandas to run data analysis on the set
- Using Matplotlib to run bar chart and pie chart graphics
- Using NumPy to apply the Haversine formula to calculate distance between two points

#### Step 1 - Import Libraries

In [ ]:
#Cell 1: Import Libraries

import pandas as pd
import matplotlib.pyplot as plt

#### Step 2 - Load and Preview the Dataset

Here, load the Pennsylvania FM dataset and take a look at the first few rows.

In [ ]:
# Load the dataset
df = pd.read_csv('fcc_db_pa_selected.csv')

# Display the first few rows of the dataset


#### Step 3: Example Analysis with PA FM Dataset

In [ ]:
# Check the shape of the dataset (rows, columns)

# Display column names

#Checking Data Types

# Summary statistics for numerical columns

Suppose we want to focus on certain columns, like frequency and city. This example shows how to select those columns.

In [ ]:
df_selected = df[['frequency', 'city']]
df_selected.head()

# Select rows by position
row_5 = df.iloc[5]

# Select rows by label
specific_row = df.loc[df['frequency'] == 88.5]

Filtering data can be useful to focus on specific frequency ranges. Let’s filter for frequencies above 107.7 MHz.

In [ ]:
# Filter stations with frequencies above 107.7 MHz


In this cell, we’ll group the data by the format column to find out how many stations are in each format category. This type of analysis can give insights into the prevalence of different programming formats.

In [ ]:
# Group by 'format' and count the number of stations for each format

Next, let’s calculate the average frequency for each format type, which might reveal trends, such as certain formats preferring higher or lower frequencies.

In [ ]:
# Group by 'format' and calculate the average frequency for each

For a comprehensive summary, we can create a DataFrame that combines both the count of stations and the average frequency by format.

In [ ]:
# Combine count and average frequency by format into a single DataFrame
format_summary = pd.DataFrame({
    'Station Count': format_counts,
    'Average Frequency (MHz)': avg_frequency_by_format
})
format_summary


This example uses Matplotlib to plot a simple bar chart.

In [ ]:
# Plotting the distribution of station formats
format_counts.plot(kind='bar', figsize=(10, 6))
plt.title("Distribution of Station Formats")
plt.xlabel("Format")
plt.ylabel("Number of Stations")
plt.show()

Counting Stations by Market

In [ ]:
# Count the number of stations for each market
market_counts = df['market'].value_counts()
market_counts

In [ ]:
count_table = df.pivot_table(values='station_name', index='market', columns='format', aggfunc='count')
count_table

Now, we’ll use Seaborn’s heatmap function to visualize count_table.

In [ ]:
import seaborn as sns

# Plotting the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(count_table, annot=True, cmap="YlGnBu", fmt=".0f", cbar_kws={'label': 'Station Count'})
plt.title("Number of Stations by Market and Format")
plt.xlabel("Format")
plt.ylabel("Market")
plt.show()

In [ ]:
# Check for missing values
print("Missing Values:\n", df.isnull().sum())

# Example: Fill missing slogan values with Unknown
df['slogan'].fillna("Unknown", inplace=True)

In [ ]:
# Add a new column 'signal_range' based on a calculation with frequency
df['signal_range'] = df['frequency'] * 2  # Just a hypothetical calculation
df.head()

# Comparing Station Count Across Market Tiers

For additional analysis, we can categorize markets into tiers based on station count, such as high, medium, and low station counts. This can be done using conditional logic.

In [ ]:
# Define thresholds for market tiers
high_threshold = 20  # markets with 20+ stations
medium_threshold = 10  # markets with 10-19 stations

# Categorize markets by station count, handling NaN values
df['market_tier'] = df['market'].map(
    lambda x: 'Unknown' if pd.isna(x) 
    else ('High' if market_counts[x] >= high_threshold 
          else ('Medium' if market_counts[x] >= medium_threshold else 'Low'))
)

# Verify the result
df[['market', 'market_tier']].head()

# Display the count of each market tier
market_tier_counts = df['market_tier'].value_counts()
market_tier_counts

Finally, visualize the distribution of market tiers to give us a more aggregated view of the data.

In [ ]:
# Plotting the market tier distribution
market_tier_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140, figsize=(8, 8))
plt.title("Distribution of Markets by Station Tier")
plt.ylabel("")  # Hide the y-label for a cleaner pie chart
plt.show()


#### Step-by-Step Guide for Calculating Distance Using the Haversine Formula

First, add a function to calculate the distance between two sets of latitude and longitude coordinates using the Haversine formula.

In [ ]:
import numpy as np

# Define the Haversine formula function
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

Next, apply this function to each row in the DataFrame to calculate the distance from Villanova University.

In [ ]:
# Villanova University coordinates
villanova_lat = 40.0379
villanova_lon = -75.3412

# Calculate the distance from each station to Villanova University
df['distance_to_villanova_km'] = df.apply(
    lambda row: haversine(villanova_lat, villanova_lon, row['lat'], row['lon']),
    axis=1
)

# Display the first few rows to verify
df[['lat', 'lon', 'distance_to_villanova_km']].head()

# Lab Exercises

1. In our Pennsylvania dataset, the longitude for each radio station is positive.  We need to change this to a negative value.  Overwrite the existing column, df['lon'] by changing each value to negative.

2. Reapply the haversine function to the df['distance_to_villanova_km'] column.

3. Create a new column that converts distance_to_villanova_km to miles.  A simple conversion formula involves taking the kilometers value and multiplying it by 0.621371 to convert it to miles.

4. Print the 10 Closest Radio Stations to Villanova.  Call the dataframe closest_10_stations

5. Use Matplotlib to plot the 10 Closest Stations to Villanova

In [ ]:
# Plotting the 10 closest stations by distance
plt.figure(figsize=(10, 6))
plt.barh(closest_10_stations['station_name'], closest_10_stations['distance_to_villanova_miles'])
plt.xlabel("Distance to Villanova (miles)")
plt.ylabel("Station Name")
plt.title("10 Closest Radio Stations to Villanova University")
plt.gca().invert_yaxis()  # Invert y-axis for closest stations at the top
plt.show()

6. The 50 Closest Stations to Villanova, Sorted by Format

In [ ]:
# Get the 50 closest stations and sort them by 'format'
closest_50_stations = df.sort_values(by='distance_to_villanova_miles').head(50).sort_values(by='format')
closest_50_stations[['station_name', 'distance_to_villanova_miles', 'format']]

7. Plot the Formats of the 50 Closest Stations in a Pie Chart

In [ ]:
# Count the number of stations per format among the 50 closest
format_counts = closest_50_stations['format'].value_counts()

# Plotting the formats in a pie chart
plt.figure(figsize=(8, 8))
plt.pie(format_counts, labels=format_counts.index, autopct='%1.1f%%', startangle=140)
plt.title("Distribution of Formats Among the 50 Closest Stations")
plt.show()